<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/banner_IA.png"  width="1000px" height="200px">

# **Taller 07:  Deep learning**

## **Outline**

1. [Ejercicio 1.](#eje1)
2. [Ejercicio 2.](#eje2)
3. [Ejercicio 3.](#eje3)
4. [Ejercicio 4.](#eje4)
5. [Ejercicio 5.](#eje5)



In [1]:
#@title **Execute this cell**
#@markdown Please include your student id
import sys
import inspect

group_id = "IA1-20252-G1" #@param {type:"string"}
assignment_id = group_id +'.taller_deep_learning'
student_id = "2225112" #@param {type:"string"}
"""
Put your student ID here

Example: student_id =  '2152145'
"""

"\nPut your student ID here\n\nExample: student_id =  '2152145'\n"

In [2]:
 #@title **Execute this cell**
#@markdown **UTILS**
#@markdown Please dont modify any line in this cell

import os
import json
import requests
from collections import namedtuple


Config = namedtuple('Config', ['server_name'])
config = Config(server_name='https://bivlabgrader.azurewebsites.net/api')


def check_solution_and_evaluate(assignment_id: str, student_func_str: str):

    # Set the endpoint and payload.
    payload = {
        'func_str': student_func_str,
        'assignment_id': assignment_id,
        'student_id': student_id
    }
    endpoint_url = config.server_name + '/CheckAndEvaluateSolution'
    # print(endpoint_url)

    # Make request to server with the data coming from the notebook.
    r = requests.post(endpoint_url, params=payload)
    pprint_json_response(r.json())
    return r


def pprint_json_response(response, indent=0):
    """Pretty print the response."""
    for key, value in response.items():
        print('\t' * indent + str(key.capitalize()))

        # If dictionary, do a recurrent call.
        if isinstance(value, dict):
            pprint_json_response(value, indent + 1)
        else:
            # Enumerate elements if list.
            if isinstance(value, list):
                if len(value) == 1:
                    print('\t' * (indent + 1) + str(value[0]))
                else:
                    for i, e in enumerate(value, start=1):
                        print('\t' * (indent + 1) + f'{i}. {e}')
            else:
                print('\t' * (indent + 1) + str(value))

In [3]:
#@title **Import libraries**

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(21)

import warnings
warnings.filterwarnings('ignore')

---
# **Ejercicio 1**  <a name="eje1"></a>
---

# Deep learning: classification

## Contexto:
Considere el dataset `penguins size` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/penguins_size.csv) el cual contiene datos sobre las dimensiones de una población de pinguinos.

In [4]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


## Tu tarea
Implemente una función que **reciba** un dataset (`df`) y que:
- **Elimine** los valores nulos del dataset.
- Convierta las columnas categóricas `["species"], ["island"], ["sex"]` a numéricas.
- Considere a la columna `["sex"]` como el ground truth.
- Determine el número de clases del dataset (`nc`)
- Destine un 90% del dataset para el proceso de entrenamiento, usando también el parámetro `random_state=21`
- Entrene una red neuronal densa con:
  - Una capa densa con 256 unidades y activación relu
  - Una capa densa con 128 unidades y activación relu
  - Una capa densa con 64 unidades y activación relu
  - Una capa densa con activación sigmoide (asigne el número de unidades según lo visto para problemas de clasificación)
- Compile el modelo con parámetros: `optimizer=tf.keras.optimizers.SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy']`
- Entrene durante 10 `epochs`.
- **Devuelva** el número de clases (`nc`)
- **Devuelva** la pérdida del modelo
- **Devuelva** el accuracy el modelo

<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [10]:
#@title **code student**

def taller07_20252_p01(df):
  import pandas as pd
  import numpy as np
  from sklearn.model_selection import train_test_split
  import tensorflow as tf
  from tensorflow import keras
  tf.random.set_seed(21)
  tf.keras.utils.set_random_seed(21)
  np.random.seed(21)

  # preprocess columns
  df = df.dropna()
  df["species"] = df["species"].factorize()[0]
  df["island"] = df["island"].factorize()[0]
  df["sex"] = df["sex"].factorize()[0]
  
  y = df["sex"]
  X = df.drop(columns=["sex"])

  #DON'T DELETE!*********************
  # Remove column names
  X.columns = range(X.shape[1])
  X = X.to_numpy()
  y = y.to_numpy()
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=21)
  #**********************************

  nc = len(np.unique(y))
  
  model = keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(nc, activation='sigmoid')
  ])
  
  model.compile(optimizer=tf.keras.optimizers.SGD(),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=10)
  
  test_loss, test_acc = model.evaluate(X_test, y_test)
  

  return nc, test_loss, test_acc

In [11]:
#@title **check your answer**
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
taller07_20252_p01(df)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4733 - loss: 984545024.0000
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5100 - loss: 1.0805 
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5100 - loss: 1.0657 
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5100 - loss: 1.0518 
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5100 - loss: 1.0388 
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5100 - loss: 1.0265 
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5100 - loss: 1.0150 
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5100 - loss: 1.0041 
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5100 - loss: 0.9939 
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5100 - loss: 0.9842 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3529 - loss: 0.9815 


(3, 0.9815411567687988, 0.3529411852359772)

## ✅ Salidas esperadas

(3, 0.9815411567687988, 0.3529411852359772)


In [12]:
#@title **send your answer**
student_func_str = inspect.getsource(taller07_20252_p01)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all outputs are correct.


---
# **Ejercicio 2**  <a name="eje2"></a>
---

# Deep learning: classification

## Contexto
El dataset "80 Cereals" (https://www.kaggle.com/datasets/crawford/80-cereals) contiene información sobre distintas marcas de cereales, sus fabricantes, información nutricional, entre otros.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1DtVWAAUMDzVkh0NqDQT0i1JvTD8B4txv")
df.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


## Tu Tarea
Desarrolle una función que **reciba** un dataset (`df`) y que:
- Elimine los valores nulos del dataset.
- Elimine la columna `["name"]`
- Convierta las columnas categóricas `["mfr"]`, `["type"]` a numéricas.
- Considere como ground truth a la columna `["type"]` (tipo de cereal).
- Destine un 85% del dataset para el proceso de entrenamiento, usando también el parámetro `random_state=21`

- Determine el número de clases del dataset (`nc`)
- Entrene una red neuronal densa con:
  - Dos (2) capas densas con 128 unidades y activación relu
  - Una capa densa con 32 unidades y activación relu
  - Una capa densa con activación sigmoide (asigne el número de unidades según lo visto para problemas de clasificación)
- Compile el modelo con parámetros: `optimizer=tf.keras.optimizers.SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy']`
- Entrene durante 10 `epochs`.
- **Devuelva** el número de clases (`nc`)
- **Devuelva** la pérdida del modelo
- **Devuelva** el accuracy el modelo




<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [13]:
#@title **code student**

def taller07_20252_p02(df):
  import pandas as pd
  import numpy as np
  from sklearn.model_selection import train_test_split
  import tensorflow as tf
  from tensorflow import keras
  tf.random.set_seed(21)
  tf.keras.utils.set_random_seed(21)
  np.random.seed(21)

  # preprocess columns
  df = df.dropna()
  df = df.drop(columns=["name"])
  
  df["mfr"] = df["mfr"].factorize()[0]
  df["type"] = df["type"].factorize()[0]
  
  y = df["type"]
  X = df.drop(columns=["type"])
  
  #DON'T DELETE!*********************
  # Remove column names
  X.columns = range(X.shape[1])
  X = X.to_numpy()
  y = y.to_numpy()
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=21)
  #**********************************

  nc = len(np.unique(y))

  model = keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(nc, activation='sigmoid')
  ])

  model.compile(optimizer=tf.keras.optimizers.SGD(),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  model.fit(X_train, y_train, epochs=10)

  test_loss, test_acc = model.evaluate(X_test, y_test)

  return nc, test_loss, test_acc

In [14]:
#@title **check your answer**
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1DtVWAAUMDzVkh0NqDQT0i1JvTD8B4txv")
taller07_20252_p02(df)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5385 - loss: 12.1557
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9692 - loss: 3.7544    
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9692 - loss: 1.7578    
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9538 - loss: 0.6549    
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9692 - loss: 34.2515   
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9692 - loss: 1.9764    
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9692 - loss: 0.3609
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9538 - loss: 0.1283
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9692 - loss: 0.0944
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9846 - loss: 0.0655 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.9167 - loss: 0.5299


(2, 0.5299080014228821, 0.9166666865348816)

## ✅ Salidas esperadas

(2, 0.529904842376709, 0.9166666865348816)


In [15]:
#@title **send your answer**
student_func_str = inspect.getsource(taller07_20252_p02)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all outputs are correct.


---
# **Ejercicio 3**  <a name="eje3"></a>
---

# Deep learning: classification

## Contexto
El dataset `supermarket sales` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/supermarket_sales.csv) contiene información relacionada con productos, costos, ganancias y otros aspectos de un supermercado.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


## Tu tarea

Implementar una función que **reciba** como parámetro el dataset (`df`), y que:
- Elimine las columnas `["Invoice ID", "Date", "Time"]`
- Convierta las columnas categóricas `["Branch"], ["City"], ["Customer type"], ["Gender"], ["Product line"], ["Payment"]` a numéricas.
- El ground truth será la columna `["Branch"]`
- Utilice un 80% del dataset para entrenamiento, con parámetros `test_size=0.2, random_state=21`)
- Determine el número de clases del dataset (`nc`)
- Entrene una red neuronal densa con:
  - Dos (3) capas densas con 256 unidades y activación relu
  - Una capa densa con activación sigmoide (asigne el número de unidades según lo visto para problemas de clasificación)
- Compile el modelo con parámetros: `optimizer=tf.keras.optimizers.SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy']`
- Entrene durante 10 `epochs`.
- **Devuelva** el número de clases (`nc`)
- **Devuelva** la pérdida del modelo
- **Devuelva** el accuracy el modelo






<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [18]:
#@title **code student**

def taller07_20252_p03(df):
  import pandas as pd
  import numpy as np
  from sklearn.model_selection import train_test_split
  import tensorflow as tf
  from tensorflow import keras
  tf.random.set_seed(21)
  tf.keras.utils.set_random_seed(21)
  np.random.seed(21)

  # preprocess columns
  df = df.drop(columns=["Invoice ID", "Date", "Time"])

  for column in ["Branch", "City", "Customer type", "Gender", "Product line", "Payment"]:
    df[column] = df[column].factorize()[0]
    
  y = df["Branch"]
  X = df.drop(columns=["Branch"])
  
  #DON'T DELETE!*********************
  # Remove column names
  X.columns = range(X.shape[1])
  X = X.to_numpy()
  y = y.to_numpy()
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=21)
  #**********************************

  nc = len(np.unique(y))
  
  model = keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(nc, activation='sigmoid')
  ])
  
  model.compile(optimizer=tf.keras.optimizers.SGD(),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=10)
  
  test_loss, test_acc = model.evaluate(X_test, y_test)

  return nc, test_loss, test_acc

In [19]:
#@title **check your answer**
import numpy as np
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
taller07_20252_p03(df)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3175 - loss: 200.7054
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3113 - loss: 1.0989
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3438 - loss: 1.0976 
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3438 - loss: 1.0973 
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3425 - loss: 1.0969 
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3450 - loss: 1.0969 
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3462 - loss: 1.0967 
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3438 - loss: 1.0969 
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3438 - loss: 1.0966
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3438 - loss: 1.0973 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3100 - loss: 1.0972  


(3, 1.0972102880477905, 0.3100000023841858)

## ✅ Salidas esperadas

(3, 1.0972102880477905, 0.3100000023841858)


In [20]:
#@title **send your answer**
student_func_str = inspect.getsource(taller07_20252_p03)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all outputs are correct.


---
# **Ejercicio 4**  <a name="eje4"></a>
---

# Deep learning: regression

## Contexto
El dataset `supermarket sales` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/supermarket_sales.csv) contiene información relacionada con productos, costos, ganancias y otros aspectos de un supermercado.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


## Tu tarea

Implementar una función que **reciba** como parámetro el dataset (`df`), y que:
- Elimine las columnas `["Invoice ID", "Date", "Time", "gross margin percentage"]`
- Convierta las columnas categóricas `["Branch"], ["City"], ["Customer type"], ["Gender"], ["Product line"], ["Payment"]` a numéricas.
- El ground truth será la columna `["Unit price"]`
- Utilice un 90% del dataset para entrenamiento, con parámetros `test_size=0.1, random_state=21`)

- Entrene una red neuronal densa con:
  - Una capa densa con 128 unidades y activación relu
  - Dos (2) capas densas con 256 unidades y activación relu
  - Una capa densa (asigne el número de unidades según lo visto para problemas de <font color="red">regresión</font>)
- Compile el modelo con parámetros: `optimizer=tf.keras.optimizers.SGD(),
              loss='mae',
              metrics=['mae','mse']`
- Entrene durante 10 `epochs`.
- **Devuelva** la pérdida del modelo



<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [23]:
#@title **code student**

def taller07_20252_p04(df):
  import pandas as pd
  import numpy as np
  from sklearn.model_selection import train_test_split
  import tensorflow as tf
  from tensorflow import keras
  tf.random.set_seed(21)
  tf.keras.utils.set_random_seed(21)
  np.random.seed(21)

  # preprocess columns
  df = df.drop(columns=["Invoice ID", "Date", "Time", "gross margin percentage"])

  for column in ["Branch", "City", "Customer type", "Gender", "Product line", "Payment"]:
    df[column] = df[column].factorize()[0]

  y = df["Unit price"]
  X = df.drop(columns=["Unit price"])

  #DON'T DELETE!*********************
  # Remove column names
  X.columns = range(X.shape[1])
  X = X.to_numpy()
  y = y.to_numpy()
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=21)
  #**********************************

  model = keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.SGD(),
               loss='mae',
              metrics=['mae','mse'])

  model.fit(X_train, y_train, epochs=10)
  test_loss, test_mae, test_mse = model.evaluate(X_test, y_test)
  return test_loss, test_mae, test_mse

In [24]:
#@title **check your answer**
import numpy as np
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
taller07_20252_p04(df)

Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 314.2537 - mae: 314.2537 - mse: 2502803.7500
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 54.8041 - mae: 54.8041 - mse: 3707.4487 
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 53.8883 - mae: 53.8883 - mse: 3608.3711 
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 52.3824 - mae: 52.3824 - mse: 3448.9966 
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49.0885 - mae: 49.0885 - mse: 3117.7927 
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38.0681 - mae: 38.0681 - mse: 2109.8640 
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24.6157 - mae: 24.6157 - mse: 848.7556
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24.5012 - mae: 24.5012 - mse: 871.1855 
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 23.1746 - mae: 23.1746 - mse: 712.8754 
Epoch 10/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23.1757 - mae: 23.1757 - mse: 712.8533 
4/4 ━━━━━━

(22.8461971282959, 22.8461971282959, 697.533447265625)

## ✅ Salidas esperadas

[22.8461971282959, 22.8461971282959, 697.533447265625]


In [25]:
#@title **send your answer**
student_func_str = inspect.getsource(taller07_20252_p04)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all outputs are correct.


---
# **Ejercicio 5**  <a name="eje5"></a>
---

# Deep learning: regression

## Contexto:
Considere el dataset `penguins size` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/penguins_size.csv) el cual contiene datos sobre las dimensiones de una población de pinguinos.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


## Tu tarea
Implemente una función que **reciba** un dataset (`df`) y que:
- **Elimine** los valores nulos del dataset.
- Convierta las columnas categóricas `["species"], ["island"], ["sex"]` a numéricas.
- Considere a la columna `["flipper_length_mm"]` como el ground truth.

- Utilice un 90% del dataset para entrenamiento, con parámetros `test_size=0.1, random_state=21`)

- Entrene una red neuronal densa con:
  - Dos (2) capas densas con 128 unidades y activación relu
  - Dos (2) capas densas con 256 unidades y activación relu
  - Una capa densa (asigne el número de unidades según lo visto para problemas de <font color="red">regresión</font>)
- Compile el modelo con parámetros: `optimizer=tf.keras.optimizers.SGD(),
              loss='mae',
              metrics=['mae','mse']`
- Entrene durante 10 `epochs`.
- **Devuelva** la pérdida del modelo



<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [26]:
#@title **code student**

def taller07_20252_p05(df):
  import pandas as pd
  import numpy as np
  from sklearn.model_selection import train_test_split
  import tensorflow as tf
  from tensorflow import keras
  tf.random.set_seed(21)
  tf.keras.utils.set_random_seed(21)
  np.random.seed(21)

  # preprocess columns
  df = df.dropna()
  for column in ["species", "island", "sex"]:
    df[column] = df[column].factorize()[0]
  
  y = df["flipper_length_mm"]
  X = df.drop(columns=["flipper_length_mm"])

  #DON'T DELETE!*********************
  # Remove column names
  X.columns = range(X.shape[1])
  X = X.to_numpy()
  y = y.to_numpy()
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=21)
  #**********************************

  model = keras.Sequential([  
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.SGD(),
               loss='mae',
              metrics=['mae','mse'])

  model.fit(X_train, y_train, epochs=10)
  test_loss, test_mae, test_mse = model.evaluate(X_test, y_test)
  return test_loss, test_mae, test_mse

In [27]:
#@title **check your answer**
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
taller07_20252_p05(df)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2378680276278675614736390824132608.0000 - mae: 2378680276278675614736390824132608.0000 - mse: inf
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 200.9347 - mae: 200.9347 - mse: 40573.4766 
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 200.8347 - mae: 200.8347 - mse: 40533.3086 
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 200.7346 - mae: 200.7346 - mse: 40493.1484 
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 200.6347 - mae: 200.6347 - mse: 40453.0117 
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 200.5347 - mae: 200.5347 - mse: 40412.8945 
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 200.4347 - mae: 200.4347 - mse: 40372.7969 
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 200.3347 - mae: 200.3347 - mse: 40332.7227 
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 200.2346 - mae: 200.2346 - mse: 40292.6641 
Epoch 10/10
10/10 ━━━━━━━━━━

(200.05999755859375, 200.05999755859375, 40196.35546875)

## ✅ Salidas esperadas

[200.05999755859375, 200.05999755859375, 40196.3515625]


In [28]:
#@title **send your answer**
student_func_str = inspect.getsource(taller07_20252_p05)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all outputs are correct.


---
<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/bannerThanks.jpg" alt="Drawing" style="width:700px;"/>